In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchtext
from torchtext.data import Field, BucketIterator
from torchtext.datasets import UDPOS
# from torchcrf import CRF

from tqdm import tqdm
import spacy
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
spacy_en = spacy.load("en_core_web_sm")

In [ ]:
def tokenizer_en(text):
    return [token.text for token in spacy_en.tokenizer(text.lower())]

In [ ]:
train_deft_file = "../deft_corpus/data/deft_files/train/t1_biology_0_0.deft"
with open(train_deft_file,'r') as fl:
    lines = fl.read().split("\n\n")

In [ ]:
lines[0].split('\n')

In [ ]:
# TOKEN TXT_SOURCE_FILE START_CHAR END_CHAR TAG TAG_ID ROOT_ID RELATION

In [ ]:
def get_text_labels(sequence_tags):
    # TOKEN TXT_SOURCE_FILE START_CHAR END_CHAR TAG TAG_ID ROOT_ID RELATION
    text = [data[0].strip() for data in sequence_tags]
    tags = [data[4].strip() for data in sequence_tags]
    return {"text":text, "labels":tags}
def parse_deft(deft_file):
    with open(deft_file, 'r') as deft:
        all_text = deft.read()
    all_sequences = []
    for lines in all_text.split("\n\n"):
        sents = []
        for token_data in lines.split("\n"):
            if len(token_data.split("\t"))==8:
                sents.append(token_data.split("\t"))
        all_sequences.append(get_text_labels(sents))
    return all_sequences
parse_deft(train_deft_file)


In [ ]:
def get_text_labels(sequence_tags):
    # TOKEN TXT_SOURCE_FILE START_CHAR END_CHAR TAG TAG_ID ROOT_ID RELATION
    text = [data[0].strip for data in sequence_tags]
    tags = [data[4].strip() for data in sequence_tags]
    return {"text":text, "labels":tags}
get_text_labels(parse_deft(train_deft_file)[1])

In [ ]:

TEXT = Field(
    sequential=True, 
    tokenize=tokenizer_en,
    init_token="<sos>",
    eos_token="<eos>",
#     batch_first=True,
    lower=True
)
LABELS = Field(
    sequential=True, 
    init_token="<sos>",
    eos_token="<eos>",
#     batch_first=True,
    is_target=True
)

In [ ]:
from torchtext.data import TabularDataset

tv_datafields = [("text", TEXT), ("labels", LABEL)]
trn, vld = TabularDataset.splits(
               path="../deft_corpus/data/Task1_folds/", # the root directory where the data lies
               train='train_0.csv',validation="val_0.csv",
               format='tsv',
               skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=tv_datafields)